# **<center><span style= "color:#00827F;">Pipelining in machine learning</span></center>**

In [1]:
# imports the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
# load the dataset
titanic_data = sns.load_dataset('titanic')

# select the features and target variable
X = titanic_data[['pclass','sex','age','fare','embarked']]
y = titanic_data['survived']

In [3]:
# split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=42)

In [4]:
# define the column transformer for imputing missing values
numeric_features = ['age','fare']
categorical_features = ['pclass','sex','embarked']

numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median'))])

categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                          ('encoder', OneHotEncoder(handle_unknown='ignore'))
                                           ])

# define the preprocessor 
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

In [ ]:
# now creating a pipline with the preprocessor(step name) and randomForest classifier(model)
pipline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

In [7]:
# fit the pipline on the training data
pipline.fit(X_train,y_train)

# make predictions on the test data
y_pred = pipline.predict(X_test)

# evaluate the model wiht accuracy score
accuracy = accuracy_score(y_test,y_pred)
print("Accuracy : ", accuracy)

Accuracy :  0.7821229050279329


## ***<span style= "color:purple;">hyperparameter tuning in pipling </span>***

In [ ]:
# imports the libraries
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [9]:
# load the dataset
titanic_data = sns.load_dataset('titanic')

# select the features and target variable
X = titanic_data[['pclass','sex','age','fare','embarked']]
y = titanic_data['survived']

# split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=42)


In [11]:
# now creating a pipline with the preprocessor(step name) and randomForest classifier(model)
pipline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore')),
    ('model', RandomForestClassifier(random_state=42))
])

# define the hyperparameters to tune
hyperparameters = {
    'model__n_estimators': [100,200,300],
    'model__max_depth': [None,5,10],
    'model__min_samples_split': [2,5,10]
}

# perfrom grid search CV
grid_search = GridSearchCV(pipline, hyperparameters, cv=5)

# fit the gscv
grid_search.fit(X_train,y_train)

# get the best model
best_model = grid_search.best_estimator_

# make predictions on the test data using the model 
y_pred = best_model.predict(X_test)

# get the accuracy score
accuracy = accuracy_score(y_test,y_pred)
print("Accuracy : ", accuracy)

# printing the best hyperparameters
print("best hyperparameters : ", grid_search.best_params_)

Accuracy :  0.8212290502793296
best hyperparameters :  {'model__max_depth': None, 'model__min_samples_split': 5, 'model__n_estimators': 300}


## ***<span style= "color:purple;">selecting best model </span>***

In [15]:
# imports the libraries
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score

In [16]:
# load the dataset
titanic_data = sns.load_dataset('titanic')

# select the features and target variable
X = titanic_data[['pclass','sex','age','fare','embarked']]
y = titanic_data['survived']

# split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=42)


In [18]:
# creating list of models to evaluate the best one
models = [
    ('Random forest', RandomForestClassifier(random_state=42)),
    ('Gradient boosting',GradientBoostingClassifier(random_state=42) )
]

best_model = None
best_accuracy = 0.0

# itrarte over the model
for name, model in models:
    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore')),
        ('model', model)
    ])

    # perform cross validation
    scores = cross_val_score(pipeline, X_train, y_train, cv=5)

    # mean accuarcy
    mean_accuracy = scores.mean()

    # fit the pipline on the training data
    pipeline.fit(X_train,y_train)

    # make predictions
    y_pred = pipeline.predict(X_test)

    # print the performance matrics
    print('model : ',name)
    print('cross-validation accuracy : ',mean_accuracy)
    print('test accuracy ', accuracy)
    
    # check if the current model has the best accuarcy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = pipeline

# retrieve the best model
print("best model : ", best_model) 

model :  Random forest
cross-validation accuracy :  0.7991529597163399
test accuracy  0.8212290502793296
model :  Gradient boosting
cross-validation accuracy :  0.8061952132374668
test accuracy  0.8212290502793296
best model :  Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('encoder', OneHotEncoder(handle_unknown='ignore')),
                ('model', RandomForestClassifier(random_state=42))])
